# 08 - LangFuse Request Routing: Tối Ưu Hóa Phân Loại và Định Tuyến Yêu Cầu

## Mục tiêu học tập

Trong notebook này, chúng ta sẽ học cách sử dụng LangFuse để:
- Thiết kế pipeline phân loại và định tuyến yêu cầu khách hàng
- Theo dõi và đo lường hiệu suất phân loại
- Tối ưu hóa quy trình định tuyến thông qua phân tích dữ liệu
- Thực hiện đánh giá và cải thiện liên tục

## Giới thiệu

Trong môi trường hỗ trợ khách hàng hiện đại, việc phân loại và định tuyến yêu cầu một cách chính xác và nhanh chóng là rất quan trọng. Các thách thức chính bao gồm:

- **Đa dạng kênh**: Email, chat, ticket, điện thoại
- **Phân loại phức tạp**: Nhiều loại yêu cầu với mức độ ưu tiên khác nhau
- **Định tuyến thông minh**: Chuyển đến đội ngũ hoặc hệ thống phù hợp
- **Đo lường hiệu suất**: Theo dõi độ chính xác và thời gian phản hồi

LangFuse giúp chúng ta giải quyết những thách thức này bằng cách cung cấp:
- Tracing chi tiết cho từng bước trong pipeline
- Đánh giá và phản hồi từ người dùng
- Phân tích hiệu suất và tối ưu hóa
- Dashboard trực quan để theo dõi

## Cài đặt và Cấu hình

In [ ]:
import os
from dotenv import load_dotenv
import json
from datetime import datetime
from typing import Dict, List, Optional, Tuple
from enum import Enum
import random
import time

# LangChain imports
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_anthropic import ChatAnthropic

# LangFuse imports
from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from langfuse.decorators import observe, langfuse_context

# Load environment variables
load_dotenv()

# Verify required environment variables
required_vars = ['LANGFUSE_SECRET_KEY', 'LANGFUSE_PUBLIC_KEY', 'ANTHROPIC_API_KEY']
for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"Environment variable {var} is required")

print("✅ Environment variables loaded successfully!")

In [ ]:
# Initialize LangFuse
langfuse = Langfuse(
    secret_key=os.getenv('LANGFUSE_SECRET_KEY'),
    public_key=os.getenv('LANGFUSE_PUBLIC_KEY'),
    host='http://localhost:3000'
)

# Initialize LLM
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0.1,
    anthropic_api_key=os.getenv('ANTHROPIC_API_KEY')
)

print("✅ LangFuse and Anthropic Claude initialized successfully!")

## Định nghĩa Hệ thống Phân loại và Định tuyến

In [ ]:
# Define request categories and routing rules
class RequestCategory(Enum):
    TECHNICAL_SUPPORT = "technical_support"
    BILLING_INQUIRY = "billing_inquiry"
    SALES_QUESTION = "sales_question"
    PRODUCT_FEEDBACK = "product_feedback"
    COMPLAINT = "complaint"
    GENERAL_INQUIRY = "general_inquiry"

class Priority(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    URGENT = "urgent"

class RoutingDestination(Enum):
    TECH_SUPPORT_TEAM = "tech_support_team"
    BILLING_DEPARTMENT = "billing_department"
    SALES_TEAM = "sales_team"
    PRODUCT_TEAM = "product_team"
    MANAGEMENT = "management"
    GENERAL_SUPPORT = "general_support"

# Define routing rules
ROUTING_RULES = {
    RequestCategory.TECHNICAL_SUPPORT: {
        "destination": RoutingDestination.TECH_SUPPORT_TEAM,
        "sla_hours": 4,
        "escalation_threshold": Priority.HIGH
    },
    RequestCategory.BILLING_INQUIRY: {
        "destination": RoutingDestination.BILLING_DEPARTMENT,
        "sla_hours": 24,
        "escalation_threshold": Priority.URGENT
    },
    RequestCategory.SALES_QUESTION: {
        "destination": RoutingDestination.SALES_TEAM,
        "sla_hours": 8,
        "escalation_threshold": Priority.HIGH
    },
    RequestCategory.PRODUCT_FEEDBACK: {
        "destination": RoutingDestination.PRODUCT_TEAM,
        "sla_hours": 48,
        "escalation_threshold": Priority.URGENT
    },
    RequestCategory.COMPLAINT: {
        "destination": RoutingDestination.MANAGEMENT,
        "sla_hours": 2,
        "escalation_threshold": Priority.MEDIUM
    },
    RequestCategory.GENERAL_INQUIRY: {
        "destination": RoutingDestination.GENERAL_SUPPORT,
        "sla_hours": 24,
        "escalation_threshold": Priority.HIGH
    }
}

print("✅ Request routing system defined successfully!")

## Thiết kế Pipeline Phân loại với LangFuse Tracing

In [ ]:
# Create classification prompt template
classification_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Bạn là một chuyên gia phân loại yêu cầu khách hàng. Nhiệm vụ của bạn là phân tích nội dung yêu cầu và phân loại chúng một cách chính xác.

Các danh mục có thể:
- technical_support: Vấn đề kỹ thuật, lỗi sản phẩm, hướng dẫn sử dụng
- billing_inquiry: Hóa đơn, thanh toán, pricing, refund
- sales_question: Thông tin sản phẩm, demo, báo giá, partnership
- product_feedback: Góp ý sản phẩm, feature request, improvement
- complaint: Phiếu nại, không hài lòng, vấn đề nghiêm trọng
- general_inquiry: Câu hỏi chung, thông tin công ty

Mức độ ưu tiên:
- low: Câu hỏi thông tin, không gấp
- medium: Yêu cầu thông thường
- high: Vấn đề ảnh hưởng business
- urgent: Vấn đề nghiêm trọng, cần xử lý ngay

Trả về kết quả dưới dạng JSON với format:
{
    "category": "<category_name>",
    "priority": "<priority_level>",
    "confidence": <float_0_to_1>,
    "reasoning": "<explanation>",
    "keywords": ["<keyword1>", "<keyword2>"]
}
    """),
    ("human", "Nội dung yêu cầu:\n\n{request_content}\n\nKênh: {channel}\nNguồn: {source}")
])

# Initialize JSON output parser
json_parser = JsonOutputParser()

print("✅ Classification prompt template created!")

In [ ]:
@observe(name="request_classification")
def classify_request(request_content: str, channel: str = "email", source: str = "website") -> Dict:
    """
    Classify customer request using LLM with LangFuse tracing
    """
    # Add metadata to current trace
    langfuse_context.update_current_trace(
        tags=["request_routing", "classification", f"channel:{channel}"],
        metadata={
            "channel": channel,
            "source": source,
            "request_length": len(request_content)
        }
    )
    
    # Create classification chain
    classification_chain = classification_prompt | llm | json_parser
    
    # Get classification result
    start_time = time.time()
    result = classification_chain.invoke({
        "request_content": request_content,
        "channel": channel,
        "source": source
    })
    classification_time = time.time() - start_time
    
    # Add classification metrics
    langfuse_context.update_current_trace(
        metadata={
            "classification_time_seconds": classification_time,
            "classified_category": result.get("category"),
            "classified_priority": result.get("priority"),
            "confidence_score": result.get("confidence")
        }
    )
    
    return result

@observe(name="request_routing")
def route_request(classification_result: Dict) -> Dict:
    """
    Route request based on classification with LangFuse tracing
    """
    category = classification_result.get("category")
    priority = classification_result.get("priority")
    
    # Get routing rules
    category_enum = RequestCategory(category)
    routing_info = ROUTING_RULES.get(category_enum, ROUTING_RULES[RequestCategory.GENERAL_INQUIRY])
    
    # Determine if escalation is needed
    priority_enum = Priority(priority)
    needs_escalation = (
        priority_enum == Priority.URGENT or
        (priority_enum == Priority.HIGH and routing_info["escalation_threshold"] in [Priority.HIGH, Priority.MEDIUM])
    )
    
    routing_result = {
        "destination": routing_info["destination"].value,
        "sla_hours": routing_info["sla_hours"],
        "needs_escalation": needs_escalation,
        "escalation_threshold": routing_info["escalation_threshold"].value,
        "routing_timestamp": datetime.now().isoformat()
    }
    
    # Update trace metadata
    langfuse_context.update_current_trace(
        tags=[f"routed_to:{routing_result['destination']}", f"priority:{priority}"],
        metadata={
            "routing_destination": routing_result["destination"],
            "sla_hours": routing_result["sla_hours"],
            "needs_escalation": needs_escalation
        }
    )
    
    return routing_result

@observe(name="complete_request_processing")
def process_customer_request(request_id: str, request_content: str, channel: str = "email", source: str = "website") -> Dict:
    """
    Complete request processing pipeline with full LangFuse tracing
    """
    # Set trace metadata
    langfuse_context.update_current_trace(
        name=f"Request Processing: {request_id}",
        tags=["customer_support", "request_processing"],
        metadata={
            "request_id": request_id,
            "channel": channel,
            "source": source,
            "processing_timestamp": datetime.now().isoformat()
        }
    )
    
    # Step 1: Classify request
    classification = classify_request(request_content, channel, source)
    
    # Step 2: Route request
    routing = route_request(classification)
    
    # Combine results
    result = {
        "request_id": request_id,
        "classification": classification,
        "routing": routing,
        "processed_at": datetime.now().isoformat()
    }
    
    return result

print("✅ Request processing pipeline created with LangFuse tracing!")

## Tạo Dữ liệu Mẫu cho Testing

In [ ]:
# Sample customer requests for testing
sample_requests = [
    {
        "id": "REQ-001",
        "content": "Chào team, tôi đang gặp lỗi khi đăng nhập vào hệ thống. Màn hình hiển thị 'Connection timeout' và không thể truy cập được. Đây là vấn đề khẩn cấp vì ảnh hưởng đến công việc của cả team.",
        "channel": "email",
        "source": "corporate_client"
    },
    {
        "id": "REQ-002", 
        "content": "Xin chào, tôi muốn hỏi về gói pricing Enterprise. Công ty chúng tôi có khoảng 500 nhân viên và cần tích hợp với hệ thống HR hiện tại. Có thể arrange một cuộc demo không?",
        "channel": "live_chat",
        "source": "website"
    },
    {
        "id": "REQ-003",
        "content": "Tôi rất không hài lòng với dịch vụ. Hóa đơn tháng này bị charge sai số tiền, đã liên hệ support 3 lần mà không ai phản hồi. Yêu cầu escalate lên management ngay!",
        "channel": "phone",
        "source": "customer_service"
    },
    {
        "id": "REQ-004",
        "content": "Hi, có thể add thêm tính năng export data ra Excel không? Hiện tại chỉ có PDF và CSV, nhưng team accounting cần format Excel để import vào hệ thống khác.",
        "channel": "email",
        "source": "feature_request"
    },
    {
        "id": "REQ-005",
        "content": "Xin chào, tôi muốn hỏi về địa chỉ văn phòng công ty và giờ làm việc. Có thể gửi thông tin liên hệ đầy đủ không?",
        "channel": "contact_form",
        "source": "website"
    },
    {
        "id": "REQ-006",
        "content": "Tôi bị charge 2 lần cho cùng một transaction. Transaction ID: TXN-20240315-001. Số tiền $299.99 bị trừ 2 lần từ card ending 4567. Cần refund ngay.",
        "channel": "email",
        "source": "billing_system"
    }
]

print(f"✅ Created {len(sample_requests)} sample requests for testing")

## Thực hiện Test Pipeline với LangFuse Monitoring

In [ ]:
# Process sample requests and collect results
processing_results = []

print("🚀 Processing sample requests...\n")

for request in sample_requests:
    print(f"Processing {request['id']}...")
    
    try:
        # Process request with full tracing
        result = process_customer_request(
            request_id=request['id'],
            request_content=request['content'],
            channel=request['channel'],
            source=request['source']
        )
        
        processing_results.append(result)
        
        # Display result summary
        classification = result['classification']
        routing = result['routing']
        
        print(f"  ✅ Category: {classification['category']}")
        print(f"  ⚡ Priority: {classification['priority']}")
        print(f"  🎯 Routed to: {routing['destination']}")
        print(f"  📊 Confidence: {classification['confidence']:.2f}")
        if routing['needs_escalation']:
            print(f"  🚨 Escalation needed!")
        print(f"  ⏱️ SLA: {routing['sla_hours']} hours\n")
        
    except Exception as e:
        print(f"  ❌ Error processing {request['id']}: {str(e)}\n")

print(f"✅ Processed {len(processing_results)} requests successfully!")

## Phân tích Kết quả và Tạo Báo cáo

In [ ]:
# Analyze processing results
def analyze_routing_performance(results: List[Dict]) -> Dict:
    """
    Analyze routing performance and generate insights
    """
    if not results:
        return {}
    
    # Category distribution
    categories = [r['classification']['category'] for r in results]
    category_counts = {}
    for cat in categories:
        category_counts[cat] = category_counts.get(cat, 0) + 1
    
    # Priority distribution  
    priorities = [r['classification']['priority'] for r in results]
    priority_counts = {}
    for pri in priorities:
        priority_counts[pri] = priority_counts.get(pri, 0) + 1
    
    # Routing destination distribution
    destinations = [r['routing']['destination'] for r in results]
    destination_counts = {}
    for dest in destinations:
        destination_counts[dest] = destination_counts.get(dest, 0) + 1
    
    # Confidence statistics
    confidences = [r['classification']['confidence'] for r in results]
    avg_confidence = sum(confidences) / len(confidences)
    min_confidence = min(confidences)
    max_confidence = max(confidences)
    
    # Escalation statistics
    escalations = [r['routing']['needs_escalation'] for r in results]
    escalation_rate = sum(escalations) / len(escalations)
    
    return {
        "total_requests": len(results),
        "category_distribution": category_counts,
        "priority_distribution": priority_counts,
        "destination_distribution": destination_counts,
        "confidence_stats": {
            "average": avg_confidence,
            "minimum": min_confidence,
            "maximum": max_confidence
        },
        "escalation_rate": escalation_rate
    }

# Generate performance report
performance_report = analyze_routing_performance(processing_results)

print("📊 ROUTING PERFORMANCE REPORT")
print("=" * 50)
print(f"Total Requests Processed: {performance_report['total_requests']}")
print()

print("📈 Category Distribution:")
for category, count in performance_report['category_distribution'].items():
    percentage = (count / performance_report['total_requests']) * 100
    print(f"  {category}: {count} ({percentage:.1f}%)")
print()

print("⚡ Priority Distribution:")
for priority, count in performance_report['priority_distribution'].items():
    percentage = (count / performance_report['total_requests']) * 100
    print(f"  {priority}: {count} ({percentage:.1f}%)")
print()

print("🎯 Routing Destination Distribution:")
for destination, count in performance_report['destination_distribution'].items():
    percentage = (count / performance_report['total_requests']) * 100
    print(f"  {destination}: {count} ({percentage:.1f}%)")
print()

print("📊 Confidence Statistics:")
stats = performance_report['confidence_stats']
print(f"  Average: {stats['average']:.3f}")
print(f"  Range: {stats['minimum']:.3f} - {stats['maximum']:.3f}")
print()

print(f"🚨 Escalation Rate: {performance_report['escalation_rate']:.1%}")
print("=" * 50)

## Thực hiện Human Feedback và Evaluation

In [ ]:
# Simulate human feedback for evaluation
@observe(name="human_feedback_evaluation")
def add_human_feedback(trace_id: str, request_id: str, feedback_data: Dict):
    """
    Add human feedback to LangFuse trace for evaluation
    """
    try:
        # Create feedback score
        langfuse.score(
            trace_id=trace_id,
            name="classification_accuracy",
            value=feedback_data['classification_score'],
            comment=feedback_data.get('classification_comment', '')
        )
        
        langfuse.score(
            trace_id=trace_id,
            name="routing_appropriateness",
            value=feedback_data['routing_score'],
            comment=feedback_data.get('routing_comment', '')
        )
        
        # Overall satisfaction
        langfuse.score(
            trace_id=trace_id,
            name="overall_satisfaction",
            value=feedback_data['overall_score'],
            comment=feedback_data.get('overall_comment', '')
        )
        
        print(f"✅ Feedback added for request {request_id}")
        
    except Exception as e:
        print(f"❌ Error adding feedback for {request_id}: {str(e)}")

# Simulate feedback data for some requests
sample_feedback = [
    {
        "request_id": "REQ-001",
        "classification_score": 1.0,  # Perfect classification
        "classification_comment": "Correctly identified as technical support with urgent priority",
        "routing_score": 1.0,
        "routing_comment": "Appropriately routed to tech support team",
        "overall_score": 1.0,
        "overall_comment": "Excellent routing decision"
    },
    {
        "request_id": "REQ-002",
        "classification_score": 1.0,
        "classification_comment": "Correctly identified as sales inquiry",
        "routing_score": 1.0,
        "routing_comment": "Perfect routing to sales team",
        "overall_score": 1.0,
        "overall_comment": "Great job on enterprise sales lead"
    },
    {
        "request_id": "REQ-003",
        "classification_score": 0.8,
        "classification_comment": "Should have been classified as both complaint AND billing inquiry",
        "routing_score": 1.0,
        "routing_comment": "Correct escalation to management",
        "overall_score": 0.9,
        "overall_comment": "Good escalation but classification could be improved"
    }
]

print("📝 Simulating human feedback collection...")
print("(In real scenario, this would come from support agents)\n")

for feedback in sample_feedback:
    print(f"Feedback for {feedback['request_id']}:")
    print(f"  📊 Classification: {feedback['classification_score']}/1.0")
    print(f"  🎯 Routing: {feedback['routing_score']}/1.0")
    print(f"  ⭐ Overall: {feedback['overall_score']}/1.0")
    print(f"  💬 Comment: {feedback['overall_comment']}\n")

print("✅ Feedback simulation completed!")

## Tối Ưu Hóa và Cải Thiện Pipeline

In [ ]:
# Advanced routing with context-aware decisions
@observe(name="enhanced_request_routing")
def enhanced_route_request(classification_result: Dict, request_metadata: Dict) -> Dict:
    """
    Enhanced routing with additional context and business rules
    """
    category = classification_result.get("category")
    priority = classification_result.get("priority")
    confidence = classification_result.get("confidence", 0.0)
    
    # Get base routing rules
    category_enum = RequestCategory(category)
    base_routing = ROUTING_RULES.get(category_enum, ROUTING_RULES[RequestCategory.GENERAL_INQUIRY])
    
    # Enhanced routing logic
    routing_result = {
        "destination": base_routing["destination"].value,
        "sla_hours": base_routing["sla_hours"],
        "needs_escalation": False,
        "escalation_reason": None,
        "routing_timestamp": datetime.now().isoformat()
    }
    
    # Business hours adjustment
    current_hour = datetime.now().hour
    is_business_hours = 9 <= current_hour <= 17
    
    if not is_business_hours and Priority(priority) in [Priority.HIGH, Priority.URGENT]:
        routing_result["needs_escalation"] = True
        routing_result["escalation_reason"] = "High priority request outside business hours"
    
    # Low confidence classification
    if confidence < 0.7:
        routing_result["needs_escalation"] = True
        routing_result["escalation_reason"] = f"Low classification confidence: {confidence:.2f}"
        routing_result["destination"] = RoutingDestination.GENERAL_SUPPORT.value
    
    # VIP customer handling
    if request_metadata.get("source") == "corporate_client":
        routing_result["sla_hours"] = max(1, routing_result["sla_hours"] // 2)  # Halve SLA for corporate
        if Priority(priority) == Priority.MEDIUM:
            routing_result["needs_escalation"] = True
            routing_result["escalation_reason"] = "Corporate client - elevated priority"
    
    # Multi-category requests (complaints with billing issues)
    keywords = classification_result.get("keywords", [])
    billing_keywords = ["charge", "billing", "refund", "payment", "invoice"]
    if category == "complaint" and any(keyword in keywords for keyword in billing_keywords):
        routing_result["additional_routing"] = RoutingDestination.BILLING_DEPARTMENT.value
        routing_result["escalation_reason"] = "Complaint with billing component - dual routing"
    
    # Update trace metadata
    langfuse_context.update_current_trace(
        tags=[
            f"enhanced_routing",
            f"confidence:{confidence:.1f}",
            f"business_hours:{is_business_hours}"
        ],
        metadata={
            "enhanced_routing": True,
            "confidence_threshold_met": confidence >= 0.7,
            "business_hours": is_business_hours,
            "vip_customer": request_metadata.get("source") == "corporate_client"
        }
    )
    
    return routing_result

print("✅ Enhanced routing system with business rules created!")

In [ ]:
# Test enhanced routing on a sample request
test_request = sample_requests[2]  # The complaint request

print("🧪 Testing Enhanced Routing...")
print(f"Request: {test_request['content'][:100]}...\n")

# First classify the request
classification = classify_request(
    test_request['content'], 
    test_request['channel'], 
    test_request['source']
)

print("📊 Classification Result:")
print(f"  Category: {classification['category']}")
print(f"  Priority: {classification['priority']}")
print(f"  Confidence: {classification['confidence']:.3f}")
print(f"  Keywords: {classification['keywords']}\n")

# Apply enhanced routing
enhanced_routing = enhanced_route_request(classification, {
    "source": test_request['source'],
    "channel": test_request['channel']
})

print("🎯 Enhanced Routing Result:")
print(f"  Destination: {enhanced_routing['destination']}")
print(f"  SLA Hours: {enhanced_routing['sla_hours']}")
print(f"  Needs Escalation: {enhanced_routing['needs_escalation']}")
if enhanced_routing.get('escalation_reason'):
    print(f"  Escalation Reason: {enhanced_routing['escalation_reason']}")
if enhanced_routing.get('additional_routing'):
    print(f"  Additional Routing: {enhanced_routing['additional_routing']}")

print("\n✅ Enhanced routing test completed!")

## Dashboard và Monitoring Insights

In [ ]:
# Create monitoring insights and recommendations
def generate_routing_insights(results: List[Dict], feedback_data: List[Dict] = None) -> Dict:
    """
    Generate actionable insights from routing performance data
    """
    insights = {
        "performance_metrics": {},
        "recommendations": [],
        "risk_indicators": [],
        "optimization_opportunities": []
    }
    
    if not results:
        return insights
    
    # Calculate performance metrics
    confidences = [r['classification']['confidence'] for r in results]
    avg_confidence = sum(confidences) / len(confidences)
    low_confidence_count = sum(1 for c in confidences if c < 0.7)
    
    escalation_count = sum(1 for r in results if r['routing']['needs_escalation'])
    escalation_rate = escalation_count / len(results)
    
    insights["performance_metrics"] = {
        "total_requests": len(results),
        "average_confidence": avg_confidence,
        "low_confidence_rate": low_confidence_count / len(results),
        "escalation_rate": escalation_rate
    }
    
    # Generate recommendations
    if avg_confidence < 0.8:
        insights["recommendations"].append({
            "type": "model_improvement",
            "priority": "high",
            "description": "Average confidence is below 80%. Consider improving classification prompts or adding more training examples."
        })
    
    if escalation_rate > 0.3:
        insights["recommendations"].append({
            "type": "process_optimization",
            "priority": "medium", 
            "description": "High escalation rate detected. Review routing rules and escalation thresholds."
        })
    
    if low_confidence_count > len(results) * 0.2:
        insights["risk_indicators"].append({
            "type": "classification_accuracy",
            "severity": "medium",
            "description": f"{low_confidence_count} requests had low confidence scores. Monitor for misclassification."
        })
    
    # Optimization opportunities
    category_counts = {}
    for result in results:
        cat = result['classification']['category']
        category_counts[cat] = category_counts.get(cat, 0) + 1
    
    most_common_category = max(category_counts, key=category_counts.get)
    if category_counts[most_common_category] > len(results) * 0.4:
        insights["optimization_opportunities"].append({
            "type": "automation",
            "category": most_common_category,
            "description": f"Consider automated handling for {most_common_category} requests (40%+ of volume)"
        })
    
    return insights

# Generate insights from our test data
routing_insights = generate_routing_insights(processing_results, sample_feedback)

print("🔍 ROUTING SYSTEM INSIGHTS")
print("=" * 50)

print("📊 Performance Metrics:")
metrics = routing_insights["performance_metrics"]
print(f"  Total Requests: {metrics['total_requests']}")
print(f"  Average Confidence: {metrics['average_confidence']:.3f}")
print(f"  Low Confidence Rate: {metrics['low_confidence_rate']:.1%}")
print(f"  Escalation Rate: {metrics['escalation_rate']:.1%}\n")

if routing_insights["recommendations"]:
    print("💡 Recommendations:")
    for rec in routing_insights["recommendations"]:
        print(f"  [{rec['priority'].upper()}] {rec['description']}")
    print()

if routing_insights["risk_indicators"]:
    print("⚠️ Risk Indicators:")
    for risk in routing_insights["risk_indicators"]:
        print(f"  [{risk['severity'].upper()}] {risk['description']}")
    print()

if routing_insights["optimization_opportunities"]:
    print("🚀 Optimization Opportunities:")
    for opp in routing_insights["optimization_opportunities"]:
        print(f"  {opp['description']}")
    print()

print("=" * 50)

## Tóm tắt và Best Practices

### Tóm tắt những gì đã học

Trong notebook này, chúng ta đã học cách sử dụng LangFuse để xây dựng và tối ưu hóa hệ thống phân loại và định tuyến yêu cầu khách hàng:

#### 🎯 **Core Components Implemented**
1. **Classification Pipeline**: Sử dụng LLM để phân loại yêu cầu theo category và priority
2. **Routing System**: Định tuyến thông minh dựa trên business rules và context
3. **LangFuse Integration**: Tracing chi tiết cho monitoring và optimization
4. **Human Feedback Loop**: Đánh giá và cải thiện liên tục
5. **Performance Analytics**: Insights và recommendations tự động

#### 📊 **Key Metrics Tracked**
- **Classification Accuracy**: Confidence scores và human feedback
- **Routing Performance**: Escalation rates và SLA compliance
- **Processing Latency**: Response times cho real-time monitoring
- **Distribution Analysis**: Category, priority, và destination patterns

#### 🔧 **Advanced Features**
- **Context-Aware Routing**: Business hours, VIP customers, multi-category requests
- **Smart Escalation**: Confidence thresholds và business rule triggers
- **Automated Insights**: Risk indicators và optimization recommendations

### Best Practices cho Production

#### 🛡️ **Reliability & Monitoring**
```python
# Always include error handling và fallback routing
try:
    classification = classify_request(content)
except Exception as e:
    # Fallback to general support với high priority
    classification = {"category": "general_inquiry", "priority": "high"}
    langfuse_context.update_current_trace(tags=["classification_error"])
```

#### 📈 **Continuous Improvement**
- **Regular Model Updates**: Retrain classification models with new data
- **A/B Testing**: Test different prompts và routing rules
- **Feedback Integration**: Actively collect và incorporate human feedback
- **Performance Baselines**: Set và monitor KPIs

#### 🔄 **Scalability Considerations**
- **Batch Processing**: Handle high-volume requests efficiently
- **Caching**: Cache common classifications để reduce latency
- **Load Balancing**: Distribute requests across multiple processing nodes
- **Queue Management**: Priority queues cho urgent requests

### Next Steps

1. **🤖 Advanced ML Integration**
   - Fine-tune models specifically cho your domain
   - Implement ensemble methods cho better accuracy
   - Add sentiment analysis cho emotional context

2. **🔗 System Integration** 
   - Connect với CRM systems
   - Integrate với ticketing platforms
   - Real-time notifications cho escalations

3. **📊 Advanced Analytics**
   - Predictive routing based on patterns
   - Customer journey analysis
   - ROI measurement cho automation

4. **🎛️ Self-Service Options**
   - Automated responses cho common requests
   - Chatbot integration với routing fallback
   - Knowledge base recommendations

## Tài liệu Tham khảo

### LangFuse Documentation
- **Tracing & Observability**: [https://langfuse.com/docs/tracing](https://langfuse.com/docs/tracing)
- **Evaluation & Scoring**: [https://langfuse.com/docs/scores-and-evaluations](https://langfuse.com/docs/scores-and-evaluations)
- **Python SDK**: [https://langfuse.com/docs/sdk/python](https://langfuse.com/docs/sdk/python)
- **Dashboard & Analytics**: [https://langfuse.com/docs/analytics](https://langfuse.com/docs/analytics)

### LangChain Integration
- **LangChain + LangFuse**: [https://langfuse.com/docs/integrations/langchain](https://langfuse.com/docs/integrations/langchain)
- **Prompt Management**: [https://langfuse.com/docs/prompts](https://langfuse.com/docs/prompts)
- **Chain Tracing**: [https://langfuse.com/docs/tracing/langchain](https://langfuse.com/docs/tracing/langchain)

### Advanced Topics
- **Custom Evaluators**: [https://langfuse.com/docs/scores-and-evaluations/custom](https://langfuse.com/docs/scores-and-evaluations/custom)
- **Datasets & Testing**: [https://langfuse.com/docs/datasets](https://langfuse.com/docs/datasets)
- **Production Best Practices**: [https://langfuse.com/docs/deployment/self-host](https://langfuse.com/docs/deployment/self-host)